# Task 5a - Calculation of $\pi$

### Goals:
- compare different integration methods for $\pi$
- determine the behaviour of the error as a function of the number of samplings for different methods
- study the autocorrelation between consecutive samplings

In [ ]:
import mc
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## Throw and count
This might be the most straightforward technique to calculate π. You throw the darts randomly at the square dart board a great many times, and then count the number of darts which fell inside the circle. This gives you an estimate of the ratio of the circle area (which is π related) to the square board area (suppose there is no dart outside the board). Mathematically, we can resort to a function $f (x)$ which defines a circle 
$$f(x, y)=\left\{\begin{array}{ll}
1, & \sqrt{x^2+y^2} \leq 1 \\
0, & \text { otherwise }
\end{array}\right.$$
where $x$ and $y$ are randomly distributed in the region [0, 1]. Then the evaluation of π is equivalent to the summation
$$ \pi=\frac{4}{N} \sum_{i=1}^N f_i(x, y) $$
where $N$ is the total number of points.

## Direct integration

The value of $\pi$ can be expressed as
$$ \pi  = 4 \int_0^2 \sqrt{1-x^2}\, \text{d}x$$
or alternatively it can be estimated using the quantity
$$A=\int_0^1\frac{1}{1+x^2}\,\text{d}x.$$
A basic Monte Carlo integration suggests that the estimate of an integral $A=\int_0^1 f(x)\,\text{d}x$ can be obtained by dividing the region [0,1] into $N$ segments, followed by a summation
$$A=\frac{1}{N} \sum_{i=1}^N f\left(x_i\right) \pm \frac{1}{\sqrt{N}} \sqrt{\left\langle f_i^2\right\rangle-\left\langle f_i\right\rangle^2}$$
where $\left\langle f_i^2\right\rangle=\sum f^2\left(x_i\right) / N,$ and  $\left\langle f_i\right\rangle=\sum_i f\left(x_i\right) / N.$ The error estimate of basic Monte Carlo integration therefore scales with $1/\sqrt{N}.$

## Importance sampling 
Very often, we find direct Monte Carlo integration is far less efficient than other integration techniques (e.g. trapezoidal or Simpson's rule) at least for one dimensional systems.
This is already quite evident from the last section.
The basic integration scheme works better when the integrand $f(x)$ is smoother.
It is thus a natural idea to replace $f(x)$ by a smoother function if it's possible.
In importance sampling method, one introduces a weight function $w(x)$ so that the integral becomes $$ A = \int_0^1 \frac{f(x)}{w(x)}w(x)\,\text{d}x.$$
If the behavior of $w(x)$ is similar to that of $f(x),$ the new integrand $g(x) = \frac{f(x)}{g(x)}$ shows much smaller variation than $f(x)$ and the Monte Carlo inegration will be more reliable for smaller $N$s.

The central issue of the scheme is to find an appropriate weight function.
For our task here, con can start with a trial function $w(x) = (4-2x)/3,$ which is properly normalised for $x\in [0,1].$
We then rewrite the original integral into $$ \tilde{A}=\int_0^1 g(x) w(x)\, \text{d} x=\int_0^1 \frac{3}{\left[1+x(y)^2\right][4-2 x(y)]}\,\text{d} y $$ where $x=2-\sqrt{4-3y}.$

In [ ]:
nsize = 10000
verbose = 1
#various Monte Carlo methods

n = nsize

task = mc.montecarlo(n, verbose=verbose)
task.sampling()
task.direct()
task.importance()

It is interesting to monitor the error $\frac{1}{\sqrt{N}} \sqrt{\left\langle f_i^2\right\rangle-\left\langle f_i\right\rangle^2}$ with respect to the number of sampling points $N.$
We can loop the direct and importance sampling schemes and save the output to a variable (`data`).

In [ ]:
#error estimate
n_init = 1000
n_final = 10000
n_step = 200
data=np.zeros((int((n_final-n_init)/n_step),5))
for i in range(n_init, n_final, n_step):
   n = i
   task = mc.montecarlo(n,verbose=0)
   task.direct()
   task.importance()
   data[int((n-n_init)/n_step), 0] = n
   data[int((n-n_init)/n_step), 1] = task.pi_1 # direct estimate
   data[int((n-n_init)/n_step), 2] = task.ds_1 # direct error
   data[int((n-n_init)/n_step), 3] = task.pi_2 # importance estimate
   data[int((n-n_init)/n_step), 4] = task.ds_2 # importance error
#    print('{0:10} {1:.10f} {2:.10f}'.format(n, task.ds_1, task.ds_2))

Now we can plot the standard deviation estimates of the error and check that they behave as $1/\sqrt{N}$.

<div class="alert alert-block alert-info"><b>TODO:</b> Plot the standard deviation estimates of the error and verify their behaviour using curve_fit </div>


In [ ]:
""" 
plot the error estimate 
and fit curve to the error estimate a*sqrt(1/n) + b
"""
fitfun = lambda x, a, b: a*np.sqrt(1/x) + b


# direct sampling
#plt.plot(... # plot the error estimate
#popt0, pcov = curve_fit(... # fit the curve
#plt.plot(... # plot the fitted curve

# importance sampling
# ...;


plt.legend()
plt.xlabel('n')
plt.ylabel('error')
plt.show()


One might also be interested in the absolute error $|\pi -4A|.$

You can plot the absolute error vs $N$ and see how direct and importance sampling integration perform for small $N\text{s}$.

Get a good estimate of $\pi$ from `np.pi`

<div class="alert alert-block alert-info"><b>TODO:</b> Plot the absolute error for direct sampling and importance sampling</div>

In [ ]:
""" compute and plot absolute error """

# direct sampling
plt.figure()
# plt.plot(... , ... , label='direct',marker='x', ls='-',
#          lw=.7,) <-------------- FILL IN

# importance sampling
# plt.plot( ... , ... ,  label='importance',marker='1', ls='-',
#          lw='.7',) <-------------- FILL IN



plt.legend()
plt.title('absolute error $|\pi-4A|$')
plt.xlabel('n')
plt.ylabel('error')
plt.show()


# Metropolis algorithm

In the last few examples, random numbers were drawn from uniform distributions in the sampling region.
As we have discussed in the importance sampling, this is not very efficient when the variation of the integrand is large.
In fact, both direct sampling and basic importance sampling result in fairly large error estimates for the $\pi$ evaluation.

Metropolis proposed in 1953 an alternative method to randomly sample the points based on the Boltzmann distribution.
Much alike the central idea in the importance sampling, we sample those points more often with higher probabilities.
But what makes Metropolis algorithm truly unique and interesting is that the sampling procedure now follows along a *Markov chain*, so that the new state depends only on the previous state.
In other words,the points in a sampling sequence are no longer randomly picked, but rather they follow the constraint:
$$ \frac{T(x\rightarrow x')}{T(x'\rightarrow x)} = \frac{w(x')}{w(x)} \geq w_i,$$
where $T(x\rightarrow x')$ is the transition rate from state $x$ to $x',$ $w(x)$ the weight function, and $w_i$ a random number in [0,1].
This equation implies *detailed balance* in statistical mechanics.
If $w(x')/w(x) \geq w_i$ we accept the new state $x'$, otherwise we reject it and use the old configuration $x$ to run another trial step.

A few remarks about the implemented Metropolis algorithm:
1. We first run the (dummy) Metropolis algorith for `nburnin` times so that the correlation with the chiuce of the initial state is reduced; this is sometimes called *burn-in* process.
2. We do not sample the data points consecutively, because it would give rise to a high correlation error.
Instead, we skip a few points in between each sampling.
3. The accepting rate, i.e. the probability of a new configuration to be accepted, will be preferred to be in the vicinity of 50%.
If the accepring rate is too high, the Metropolis algorithm behaves just like a basic importance sampling scheme.
On the other hand, when the accepting rate is very low, much of the time is wasted in searching the new configuration.
An accepting rate of around 50% usually offers a balanced performance in terms of efficiency and accuracy.
The accepting rate is intimately dependent in the step size `h`, which is used to control the searching window for the new point.


The improvement of the absolute error might not be straightforward if we compare the Metropolis scheme implemented in the class `metropolis` to the direct importance sampling methods.
However, the error of a Metropolis integration should be much smaller.
Bear in mind that we're using an elementary weight function here (same as the importance sampling), and is quite challenging to find a very good weight function.

<div class="alert alert-block alert-info"><b>TODO:</b> Change the step size `h` and see how the accepting rate responds. Does a large `h` lead to a small accepting rate?</div>

In [ ]:
#Metropolis

nskip = 1
nburnin = nsize
h = 0.9

task = mc.metropolis(nsize, nskip, nburnin, h, verbose)
task.run()


The correlation between data points can be analyzed by the *autocorrelation function*, which is defined as
$$ C(l)=\frac{\left\langle A_{n+l} A_n\right\rangle-\left\langle A_n\right\rangle^2}{\left\langle A_n^2\right\rangle-\left\langle A_n\right\rangle^2},$$
where $ \left\langle A_{n+l} A_n\right\rangle=\frac{1}{M} \sum_{n=1}^M A_{n+l} A_n $ and $M$ is `nsize` in our code.

You can check the correlation error from consecutive points by plotting the correlation function vs the lagging steps $l.$

Find the minimum number of samplings $l$ we need to skip in order to get a minimal correlation beween $A_n$ and $A_{n+l}.$

<div class="alert alert-block alert-info"><b>TODO:</b> Plot the autocorrelation function and determine the minimum value of `l` such that the correlation between two samplings is minimal.</div>

In [ ]:
#autocorrelation

# lmax = 20               # the largest l of the series
# nrep = 10               # perform some independent experiments and do an ensemble-average 
lmax = 20
nrep = 3
corr = np.zeros((lmax + 1,2))       # create a two column array to store C(l) vs. l.
corr[:,0] = np.arange(0,lmax+1,1)
for i in range(lmax+1):
   c = 0
   for j in range(nrep):
       task = mc.metropolis(nsize, i, nburnin, h, verbose)
       task.run()
       c += task.corr
   corr[i,1] = c/nrep

np.savetxt('autocorr.dat', corr)    # for plotting

In [ ]:
""" plot autocorrelation """

corr = np.loadtxt('autocorr.dat')
plt.figure()
plt.plot(corr[:,0], corr[:,1], marker='x', ls='')
plt.xlabel('l')
plt.ylabel('C(l)')
plt.title('autocorrelation')

